In [1]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime

## Group Assignment
### Team Number: 12
### Team Member Names: Sharuga, Derek, Alex
### Team Strategy Chosen: Market Meet

Disclose any use of AI for this assignment below (detail where and how you used it).  Please see the course outline for acceptable uses of AI.


In [2]:
START_DATE = '2023-11-25'
END_DATE = '2024-11-23'
INVESTMENT = 1000000


$AGN: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$CELG: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$MON: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$RTN: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


['AAPL',
 'ABBV',
 'ABT',
 'ACN',
 'AIG',
 'AMZN',
 'AXP',
 'BA',
 'BAC',
 'BB.TO',
 'BIIB',
 'BK',
 'BLK',
 'BMY',
 'C',
 'CAT',
 'CL',
 'KO',
 'LLY',
 'LMT',
 'MO',
 'MRK',
 'PEP',
 'PFE',
 'PG',
 'PM',
 'PYPL',
 'QCOM',
 'RY.TO',
 'SHOP.TO',
 'T.TO',
 'TD.TO',
 'TXN',
 'UNH',
 'UNP',
 'UPS',
 'USB']

In [ ]:
# Function to calculate the purchasing fee depending on the number of shares
def calculate_purchase_fee(shares)
    return max(3.95, 0.001*shares)

In [3]:

def validity(tickers):
    final_list = []
    for ticker in tickers:
        #only append the stock to the final stock list if it is valid
        if (not check_delist(ticker) and
            check_currency(ticker) and
            check_trading_days(ticker)):
                final_list.append(ticker)
    return final_list

def check_delist(ticker):
    stock = yf.Ticker(ticker)
    try:
        data = stock.history(period='1d')
        if data.empty:
            #if we can't find any data on the stock, it's delisted
            return True
        else:
            #check that there is actually valid market data for this stock - AI
            if 'Close' not in data.columns or data['Close'].isnull().all():
                #if there are no valid close prices, the stock is delisted
                return True
            return False
    except Exception as e:
        #if there is an error in finding the stock's data, we can assume that it's delisted
        return True

def check_trading_days(ticker):
    min_days=18
    #the stock should have at least 18 trading days (we can check this by checking the last month)
    stock = yf.Ticker(ticker)
    try:
        data = stock.history(period="1mo") #fetches the stock info from the last month
        #AI
        trading_days = data.shape[0]
        return trading_days >= min_days  # Must have at least `min_days` trading days
    except Exception as e:
        return False  # If error, assume it’s invalid

def check_currency(ticker):
    """Check if the stock is in USD or CAD currency."""
    stock = yf.Ticker(ticker)
    try:
        currency = stock.info.get('currency', None)
        return currency in ['USD', 'CAD']
    except Exception as e:
        return False  # If error, assume it’s invalid

all_data = pd.read_csv('Tickers_Example.csv', header=None)

all_data = pd.DataFrame(validity(all_data[0]))
all_data.rename(columns = {0:'Ticker'}, inplace=True)


# Passes the first (and only) column to the function
# validity(all_data[0])

$AGN: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$CELG: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$MON: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$RTN: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


In [4]:
# Create a list of all the stock betas (retrieved from yfinance) of stock tickers in a list
def get_betas(tickers):
    betas = []
    
    for ticker in tickers:
        betas.append(yf.Ticker(ticker).info.get('beta'))

    return betas

In [5]:
# Add a column containing all the stock betas to [all_data]
all_data['Beta'] = get_betas(all_data['Ticker'])

all_data.head()

,Ticker,Beta
0,AAPL,1.240
1,ABBV,0.613
2,ABT,0.722
3,ACN,1.245
4,AIG,1.069


In [15]:
# Choose the max(24, data.length) stocks that match the market the closest
def filter_betas(data):
    # Remove any negative betas, as long as data ends with at least 12 characters
    data.sort_values(by='Beta', axis=0, inplace=True, kind='quicksort')
    data.drop(index=data[(data['Beta'] <= 0) & (len(data) > 12)].index, inplace=True)
    data.reset_index(drop=True, inplace=True)

    data['temp'] = abs(1-data['Beta'])     # Add absolute difference between 1 and the beta of each stock to a new column
    data.sort_values(by='temp', axis=0, inplace=True, kind='quicksort')    # Sort data by temp
    data.drop(columns='temp', axis=1, inplace=True)    # Remove temp

    data.index = range(0, len(data))    # Reassign the index
    
    return data[:24] if len(data) > 24 else data    # Return first 24 elements of data if it is greater than 24, other wise return data

In [16]:
filter_betas(all_data)

,Ticker,Beta
0,UPS,0.982
1,TXN,0.980
2,USB,1.040
3,UNP,1.056
4,BK,1.060
5,BB.TO,1.068
6,AIG,1.069
7,CAT,1.095
8,AMZN,1.146
9,RY.TO,0.842


In [8]:
tsx60_info = yf.Ticker('XIU.TO')
tsx60 = tsx60_info.history(start='2023-10-01', end='2024-09-30', interval='1d')

sp500_info = yf.Ticker('^GSPC')
sp500 = sp500_info.history(start='2023-10-01', end='2024-09-30', interval='1d')
sp500 = sp500.reindex(tsx60.index, method='nearest')

market_avg = (tsx60['Close'].pct_change() + sp500['Close'].pct_change())/2
market_avg = market_avg.dropna()
market_avg.index = market_avg.index.strftime('%Y-%m-%d')
market_avg_mean = market_avg.mean()

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.